▶️ **Link to Youtube Video:** [Day 8 - Working with raster files in Python | Introduction to Rasterio (Part 2)](https://youtu.be/1Fa0Y7wDwDQ?si=kRdRJ1is6xO3PLro)


▶️ **Link to Full Youtube Playlist:** [12 Days Geospatial Python Bootcamp](https://youtube.com/playlist?list=PLPBWT_CJ5QhL90iN3n6zWGpSXQLw42ToU&si=04Dv0mI3pPpBK29z)

### Working with Rasters in Python (Intro to Rasterio - Part 2)

In [ ]:
import rasterio as rio

### Opening a dataset

In [ ]:
with rio.open("./dem/dem.tif") as src:
    metadata = src.meta


src.width
src.height
src.crs
src.transform
src.bounds
src.count
src.driver
src.dtypes
src.nodata

### Visualizing Rasters

In [ ]:
from rasterio.plot import show

with rio.open("./dem/dem.tif") as src:
    band = src.read(1)


show(
    band,
    # contour=True,
    title="Lake Bosomtwe DEM"
)

### Raster Histograms

In [ ]:
from rasterio.plot import show_hist

with rio.open("./dem/dem.tif") as src:
    band = src.read(1)

show_hist(band, title="Lake Bosomtwe Histogram")

### Transoforming between pixel coordinates and spatial coordinates

In [ ]:
with rio.open("./dem/dem.tif") as src:
    band = src.read(1)
    transform = src.transform

# print(src.crs)
# Pixel to Spatial Coordinates
pixel_coord = (100, 100)
spatial_coord = transform * pixel_coord
print("Pixel to Spatial Coordinates:", spatial_coord)

# Spatial Coordinate to Pixel Coordinates
spatial_coord_rev = (-1.4809722221985637, 6.5612499999929215)
pixel_coord_rev = src.index(spatial_coord_rev[0], spatial_coord_rev[1])
print("Spatial Coordinate to Pixel Coordinates:", pixel_coord_rev)


### Finding Maximum Value (Elevation, DEM)


In [ ]:
import numpy as np

with rio.open("./dem/dem.tif") as src:
    band = src.read(1)
    transform = src.transform


maximum_pixel_value = np.max(band)
maximum_pixel_index = np.unravel_index(np.argmax(band), band.shape)

print("Maximum value is:", maximum_pixel_value)
print("Maximum value index:", maximum_pixel_index)

max_point_spatial = transform * (maximum_pixel_index[0], maximum_pixel_index[1])
print("Maximum value spatial coord:", max_point_spatial)


### Selection and Indexing

In [ ]:
with rio.open("./dem/dem.tif") as src:
    band = src.read(1)
    transform = src.transform


raster_value = band[593, 481]
raster_value

### Creating Composites

In [ ]:
import os
import rasterio as rio
import numpy as np
import matplotlib.pyplot as plt

parent_dir = os.path.normpath(r"/home/tommy/12_Days_GeoPython_Training/Day_8")

"""
Create a folder inside your working directory called bands and place all your images
there
"""

# File paths
data_2 = "bands/B2.TIF" 
data_3 = "bands/B3.TIF"
data_4 = "bands/B4.TIF"
data_5 = "bands/B5.TIF"
data_6 = "bands/B6.TIF"
data_7 = "bands/B7.TIF"

# Function to open files
def read_band(file):
    path_to_file = os.path.join(parent_dir, file)
    with rio.open(path_to_file) as src:
        return src.read(1)
    
def normalize(array):
    array_min, array_max = array.min(), array.max()
    normalized = (array - array_min) / (array_max - array_min)
    return normalized

    
# Read all bands
band_2 = read_band(data_2)  # Blue
band_3 = read_band(data_3)  # Green
band_4 = read_band(data_4)  # Red
band_5 = read_band(data_5)  # NIR
band_6 = read_band(data_6)  # SWIR1
band_7 = read_band(data_7)  # SWIR2

true_color_composite = np.dstack((band_4, band_3, band_2))
normalized_composite = normalize(true_color_composite)

plt.imshow(normalized_composite)
plt.show()

### Saving a Raster

In [ ]:
with rio.open(os.path.join(parent_dir, data_2)) as src:
    profile = src.profile
    meta = src.meta

# Update metadata for 3-band image
meta.update({
    "count": 3
})

# Transpose the array to (count, height, width) format
output_data = np.transpose(normalized_composite, (2, 0, 1))

output_path = os.path.join(parent_dir, 'composite.tif')
print(output_data.shape)

with rio.open(output_path, "w", **meta) as dataset:
    dataset.write(output_data)


In [ ]:
from rasterio.plot import show_hist

with rio.open(output_path) as raster:
    count = raster.count

    show_hist(raster.read())